In [1]:
import numpy as np
import hera_cal.redcal as rc
import matplotlib.pyplot as plt
%matplotlib notebook 

In [2]:
def build_hex_array(hexNum, sep=14.7):
    antpos, i = {}, 0
    for row in range(hexNum-1,-(hexNum),-1):
        for col in range(2*hexNum-abs(row)-1):
            xPos = ((-(2*hexNum-abs(row))+2)/2.0 + col)*sep;
            yPos = row*sep*3**.5/2;
            antpos[i] = np.array([xPos, yPos, 0])
            i += 1
    return antpos

# Pseudocode
* Compute array
* Compute all baselines
* Figure out which orientations to consider
* Simulate visibilities



In [3]:
pos = build_hex_array(11)
ubl_pairs = [red[0] for red in rc.get_pos_reds(pos)]
ors = {(i,j): pos[i] - pos[j] for (i,j) in ubl_pairs}
for key in ors.keys():
    ors[key] = tuple([int(1000000000*comp)/1000000000. for comp in ors[key]/np.linalg.norm(ors[key])])

uors = {}
for ubl,o in ors.items():
    if uors.has_key(o):
        uors[o].append(ubl)
    else:
        uors[o] = [ubl]

np.sum(np.array([len(uor) for uor in uors.values()]) > 0)

388

In [34]:
import scipy.cluster
clusters = scipy.cluster.hierarchy.fclusterdata(ors.values(),1e-6, criterion='distance')
print len(clusters), np.max(clusters)

1133 384
